# Case Study 1 : Collecting Data from Twitter

Due Date: September 22, **before the beginning of class at 6:00pm**

* ------------

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    Melanie Jutras
    
    Debraj Roy
    
    Praneeth
    
    Rahul
    
    Sushma

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

# Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [10]:
import twitter
import json
import pymongo

#--------------------------------------------------------------------------------------
# The purpose of oauth_login() function is
#    1. Login to the Twitter API
#    2. Return twitter handle necessary for its use in later functions

def oauth_login():
    # Had to go to http://twitter.com/apps/new to create an app and get values
    # for these credentials
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'zmbyPowgnW1QNs9LXjGtNqMcm'
    CONSUMER_SECRET ='wnbgjQQJxidVvnw3CORSw5MVjbOE8KstdXlkPLrH01R9JLAlPn'
    OAUTH_TOKEN = '1529017262-SoyYbWBJWW8Mn0MLuyQLpmViknAiGQg1zVJv3N7'
    OAUTH_TOKEN_SECRET = '4nqZePogkTfNVBJqqb1VN0dpprz8ykeKBDrdxXsxDFYnC'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

def save_to_mongo(data, mongo_db, mongo_db_coll, **mongo_conn_kw):
    
    # Connects to the MongoDB server running on 
    # localhost:27017 by default
    
    client = pymongo.MongoClient(**mongo_conn_kw)
    
    # Get a reference to a particular database
    
    db = client[mongo_db]
    
    # Reference a particular collection in the database
    
    coll = db[mongo_db_coll]
    
    # Perform a bulk insert and  return the IDs
    
    return coll.insert_many(data)

def load_from_mongo(mongo_db, mongo_db_coll, return_cursor=False,
                    criteria=None, projection=None, **mongo_conn_kw):
    
    # Optionally, use criteria and projection to limit the data that is 
    # returned as documented in 
    # http://docs.mongodb.org/manual/reference/method/db.collection.find/
    
    # Consider leveraging MongoDB's aggregations framework for more 
    # sophisticated queries.
    
    client = pymongo.MongoClient(**mongo_conn_kw)
    db = client[mongo_db]
    coll = db[mongo_db_coll]
    
    if criteria is None:
        criteria = {}
    
    if projection is None:
        cursor = coll.find(criteria)
    else:
        cursor = coll.find(criteria, projection)

    # Returning a cursor is recommended for large amounts of data
    
    if return_cursor:
        return cursor
    else:
        return [ item for item in cursor ]



#--------------------------------------------------------------------------------------
# The purpose of the get_trends() function is:
#    1. Gather current world trends from twitter
#    2. Gather current US trends from twitter
#    3. Print the trends for viewing (TODO: Do we want to save trending data?)

def get_trends(twitter_api):
    
    # The Yahoo! Where On Earth ID for the entire world is 1.
    # See https://dev.twitter.com/docs/api/1.1/get/trends/place and
    # http://developer.yahoo.com/geo/geoplanet/

    WORLD_WOE_ID = 1
    US_WOE_ID = 23424977

    # Prefix ID with the underscore for query string parameterization.
    # Without the underscore, the twitter package appends the ID value
    # to the URL itself as a special case keyword argument.

    world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
    us_trends = twitter_api.trends.place(_id=US_WOE_ID)


    print ('*** Here are the WORLD TRENDS :')
    print (json.dumps(world_trends, indent=1))
    print ('*** Here are the US TRENDS :')
    print (json.dumps(us_trends, indent=1))

#--------------------------------------------------------------------------------------
# The purpose of the getPokemonGoTweets() function is:
#   1. Search twitter for tweets containing the search term '#PokemonGo'
#   2. Save any tweets retrieved in a file named 'pokemonresults.txt'

def getPokemonGoTweets(twitter_api):
    
       
    # Import unquote to prevent url encoding errors in next_results
    from urllib.parse import unquote

    # Set q to the topic of interest for CaseStudy1-Problem1, PokemonGo 

    q = '#PokemonGo' 

    count = 100

    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets

    search_results = twitter_api.search.tweets(q=q, count=count)

    statuses = search_results['statuses']


    # Iterate through 5 more batches of results by following the cursor

    for _ in range(5):
        print ("Length of statuses", len(statuses))
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break

        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') for kv in unquote(next_results[1:]).split("&") ])

        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        
    # Dump data to json file in json format    
    with open('pokemonresults.txt', 'w') as outfile:
        json.dump(statuses, outfile)  
    # Note: by opening file using 'with', no need to close() the file. It happens implicitly.
    

#--------------------------------------------------------------------------------------
# The purpose of the twitter_search() function is:
#   1. Search twitter for tweets containing the search term q, defaulting results=200
#      because oAuth users can "only" make 180 search queries per 15 minute interval
#
#   2. return statuses

def twitter_search(twitter_api, q, max_results=200, **kw):

    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets and 
    # https://dev.twitter.com/docs/using-search for details on advanced 
    # search criteria that may be useful for keyword arguments
    
    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets    
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
    
    statuses = search_results['statuses']
    
    # Iterate through batches of results by following the cursor until we
    # reach the desired number of results, keeping in mind that OAuth users
    # can "only" make 180 search queries per 15-minute interval. See
    # https://dev.twitter.com/docs/rate-limiting/1.1/limits
    # for details. A reasonable number of results is ~1000, although
    # that number of results may not exist for all queries.
    
    # Enforce a reasonable limit
    max_results = min(1000, max_results)
    
    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break
            
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        
        if len(statuses) > max_results: 
            break
            
    return statuses

        

#--------------------------------------------------------------------------------------
# The purpose of the mainDataGathering() function is:
#     1. Login to twitter and get api handle
#     1. Gather desired twitter data and save for future use

def mainDataGathering():
    
    twitter_api = oauth_login()
    
    print(twitter_api)
    
    #get_trends(twitter_api)

    #getPokemonGoTweets(twitter_api)
    
    # Search twitter for "#PokemonGo" tweets
    q = "#PokemonGo"
    results = twitter_search(twitter_api, q, max_results=10)
        
    # Save results to MongoDB
    save_to_mongo(results, 'search_results', q)

    print('**** Load the DATABASE ***')
    # Load results from MongoDB that contain "Drowzee"
    #q="the"
    cursor = load_from_mongo('search_results', q, return_cursor=True)
    # Show one sample search result by slicing the list...
    #print (json.dumps(search_results[0], indent=1))
    print('**** HERE is the data in the DATABASE ***')
    for trend in cursor:
#        print(trend['text'])
        print(trend['user']['location'])
    
    # OR...
    # Show one sample search result by slicing the list...
    # print (json.dumps(results[0], indent=1))
    


    

#--------------------------------------------------------------------------------------
# Call the mainDataGathering() function to perform actions needed for 
# DS501 CaseStudy 1 Problem 1 : Sampling twitter data with streaming api about a topic

mainDataGathering()

#--------------------------------------------------------------------------------------
# Test that the mainDataGathering() worked by displaying a sample result from the file
# This is not necessary and can be commented out if display is not desired here...

#print ("*** Sample PokemonGo output : ")
# Get one sample search result from file
#with open('pokemonresults.txt','r') as json_data:
#    json_statuses = json.load(json_data)
#    print(json.dumps(json_statuses[0], indent=4))
# Note: by opening file using 'with', no need to close() the file. It happens implicitly.

# q = "#PokemonGo"
#load_from_mongo('search_results', q)



**** Load the DATABASE ***
**** HERE is the data in the DATABASE ***

El Mundo
Bow(ner), London
霧弥湖町
Uddevalla, Sverige
Cardiff, Wales
Nacogdoches, TX
C H I C A G O
Philippines

Canada
Lyon, France
Jamaica
HumbleCity, MD
Los Angeles, CA

Ateneo de Davao University

hailing satan w/ nico nii.
she/her
DLSU
Laval, Québec
Yaoi World 
Strasbourg & Portugal
everywhere




Malta
Barberton, OH
Paypal: Bdawg263@Gmail.com

UK
Tauranga City, New Zealand

宮城県在住
Enschede, Nederland

Houston, TX
Wellington City, New Zealand
Toronto, Canada
In my head


México
Schwaig bei Nürnberg, Deutschland
Longmont, CO

Fairview, Dublin 3
日本 秋田県
México
San Diego, CA
Uddevalla, Sverige
Carcassonne, France


Huddersfield
Natal, Brasil
Uddevalla, Sverige
My House

Arcadia, CA
Panama/Inglaterra/Noruega


Uddevalla, Sverige
Montreal


United States


United States
Salt Lake City, UT
Tokyo, Japan
Tauranga City, New Zealand
México, DF
水道橋住。大学生やってます
Charleston, SC

Charleston, SC
Santa Bárbara d'Oeste, Brasil
Discord His

### Report some statistics about the tweets you collected 

* The topic of interest: < INSERT YOUR TOPIC HERE>


* The total number of tweets collected:  < INSERT THE NUMBER HERE>

*-----------------------

# Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [30]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

status_texts = [ status['text'] 
                 for status in statuses ]

screen_names = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

# Compute a collection of all words from all tweets
words = [ w 
          for t in status_texts 
              for w in t.split() ]

# Explore the first 5 items for each...

print json.dumps(status_texts[0:5], indent=1)
print json.dumps(screen_names[0:5], indent=1) 
print json.dumps(hashtags[0:5], indent=1)
print json.dumps(words[0:5], indent=1)

from collections import Counter

for item in [words, screen_names, hashtags]:
    c = Counter(item)
    print (c.most_common()[:30]) # top 30
    print()












**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [31]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


# Example 10










**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [32]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


# Example 14




* ------------------------

# Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [2]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


# Example 19 in Chapter 9




* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [3]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary








*------------------------

# Problem 4: Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [4]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary





*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu) *and* the TA Wen Liu (wliu3@wpi.edu).
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

** Totoal Points: 120 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API.  Come up with a business question and describe how Twitter data can help you answer that question.




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

